In [1]:
# NameNode 확인
!jps

4322 WebAppProxyServer
40 QuorumPeerMain
10570 JournalNode
2986 DFSZKFailoverController
4236 JobHistoryServer
20846 Jps
10799 NameNode
2643 JournalNode
787 Kafka
3733 ResourceManager
20505 SparkSubmit
20731 CoarseGrainedExecutorBackend
11836 NodeManager
11070 DataNode


In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

Using TensorFlow backend.


In [4]:
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import nltk

In [5]:
spark

In [6]:
spark
sc = spark.sparkContext

In [7]:
import re

def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.lower()
    text=text[((text['total_text_list'].str.len()>= 10)) & ((text['total_text_list'].eq(' ')==False) & (text['total_text_list'].eq('')==False))]
    return text
def regex2_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['text'] = text['text'].str.lower()
    text=text[((text['text'].str.len()>= 10)) & ((text['text'].eq(' ')==False) & (text['text'].eq('')==False))]
    return text

def regex_3(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

def regex_4(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    #text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

# general_minsun2.csv
general = pd.read_csv('/root/general.csv', sep=',',  lineterminator='\n')
general = general.drop_duplicates()
general = spark.createDataFrame(general[['total_text_list']])
general = general.select(col('total_text_list').alias('text')) 
general = general.withColumn('label', lit(0))
general.show(5)
print(general.count())
general = general.withColumn("label",col("label").cast("integer"))


/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

+--------------------+-----+
|                text|label|
+--------------------+-----+
|im at ap stgate i...|    0|
|why do i love thi...|    0|
|nowplaying charmi...|    0|
|im at kuwait city...|    0|
|lolim right hande...|    0|
+--------------------+-----+
only showing top 5 rows

801932


In [8]:
# 보안 관련 계정들 트윗
security = pd.read_csv('/root/dataset.csv', sep=',',  lineterminator='\n')
security = security[["text","datetime"]]
security = security.sort_values(by='datetime' ,ascending=True)
security = security.reset_index(drop=True)
security = regex2_(security)
security = security[~security['text'].str.contains("rt", na=False, case=False)]
security = security.drop_duplicates()
security = spark.createDataFrame(security[['text']])
security = security.withColumn('label', lit(1))
security = security.withColumn("label",col("label").cast("integer"))

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

In [9]:
t1 = security.collect()[200000:225000]
t5 = general.collect()[200000:225000]

t1 = spark.createDataFrame(t1)
t5 = spark.createDataFrame(t5)

df1 = t1.union(t5)

In [10]:
df1.show(3, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                                                           |label|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
| analysis malware exploit special easter edition c msecurity secrets daily   tinolle editioniddbcbeaaccutmcampaignpapersubutmmediumemailutmsourcesubscription  |1    |
|trfareita  rlxb       vdf     sun  apr                                                                                                                         |1    |
|daily analysis note upatre is back to ssl  l    malwaremustdie                                                                                                 

In [11]:
word1 = security.collect()[200000:200100]
word2 = general.collect()[200000:200100]
word1 = spark.createDataFrame(word1)
word2= spark.createDataFrame(word2)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2





class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:50000] # 24939 99756
        neg_df = result_pdf[50000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word1, word2)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(df1)
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["text"], 
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
dataset = label_str_index.fit(df_with_vectors).transform(df_with_vectors)

50000
130
5
<class 'pyspark.sql.dataframe.DataFrame'>


In [13]:
train, test = dataset.randomSplit(weights=[0.8,0.2])

In [14]:
train.show(3, truncate=False)

+---------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|text                                          

In [15]:
import re
import codecs
from tqdm import tqdm
import shutil
import pickle
from tensorflow.keras import optimizers
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D


In [16]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import keras
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

In [18]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [19]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(2, activation='softmax', name="output")(l)

In [20]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [21]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

## Option 1) Spark Model

In [22]:
X_train = train.toPandas()['text']
# X_train = np.asarray(X_train).astype(np.int)
Y_train = train.toPandas()['label']
# y_train=np.asarray(y_train).astype(np.int)

/root/spark/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [23]:
X_train

0                                       opensslufj rqrjtd 
1                         akiba pc hotline  rcooe    haxc 
2          crypto is not pervasive easy to use the defa...
3         analysis malware exploit cyberwar infosec int...
4                   dpm  dpm  bmr   mw    microsofttechnet
                               ...                        
40014       you must appreciate the little effort guurllol
40015                            you picked the wrong girl
40016                                         you too sabi
40017    your only evidence to your claim is hes messed...
40018                               youre obsessed with me
Name: text, Length: 40019, dtype: object

In [24]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(Y_train, 2)
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [25]:
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd
import time
t1 = time.time()
rdd = to_simple_rdd(sc, X_train, y_train)
spark_model = SparkModel(model, frequency='epoch', mode='asynchronous', num_workers=3, custom_objects={'KerasLayer': hub.KerasLayer})
spark_model.fit(rdd, epochs=10, batch_size=64, verbose=0, validation_split=0.1)
t2 = time.time() - t1
print(t2)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 13:24:07] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 13:24:07] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 13:24:38] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 14:07:41] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 14:07:43] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 14:15:07] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 14:15:10] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 14:15:11] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 14:15:12] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 14:52:07] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 14:52:08] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 15:05:47] "POST /update HTTP/1.1" 2

INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 19:08:03] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 19:08:04] "GET /parameters HTTP/1.1" 200 -
ERROR:werkzeug:45.227.254.54 - - [01/Nov/2022 19:10:46] code 400, message Bad HTTP/0.9 request type ('\x03\x00\x00/*à\x00\x00\x00\x00\x00Cookie:')
INFO:werkzeug:45.227.254.54 - - [01/Nov/2022 19:10:47] "  /*à     Cookie: mstshash=Administr" HTTPStatus.BAD_REQUEST -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 19:14:01] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 19:14:01] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 19:15:36] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 19:15:37] "GET /parameters HTTP/1.1" 200 -
ERROR:werkzeug:23.224.186.212 - - [01/Nov/2022 19:41:20] code 400, message Bad request syntax ('\x16\x03\x01\x00¯\x01\x00\x00«\x03\x03øsÛ<NÙðRp`¨-\x17\x81½\x82®òÚ×J\x1d\x82^\x82@¹~\x85®%\x15\x00\x00\x1aÀ/À+À\x11À\x07À\x13À\tÀ\x14À')

>>> Async training complete.


INFO:werkzeug:172.28.0.1 - - [01/Nov/2022 21:21:48] "GET /parameters HTTP/1.1" 200 -


28719.609904527664


In [28]:
X_test = test.toPandas()['text']
# X_train = np.asarray(X_train).astype(np.int)
Y_test = test.toPandas()['label']
y_test = to_categorical(Y_test, 2)
# y_train=np.asarray(y_train).astype(np.int)

In [29]:
spark_model.save('bert_1')

In [38]:
X_test

0                    drweb    doctor web pacific  uemkuj 
1        analysis malware exploit cyberwar surveillanc...
2        analysis malware exploit special easter editi...
3        malware  hunters  aprmalware  it must die   m...
4                                        nsa     cbmofug 
                              ...                        
9976    whenever there is a three or four cent price a...
9977                  who are you who are you waiting for
9978    wish i could come join you in that bed and get...
9979    with you gen in tyms of rain and drought  you ...
9980    youre really beefing an interm managerwe knew ...
Name: text, Length: 9981, dtype: object

In [37]:
t1 = time.time()
# predictions = spark_model.predict(X_test) # perform inference
evaluation = spark_model.evaluate(X_test, y_test) # perform evaluation/scoring
t2 = time.time() - t1
print(t2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 26.0 failed 4 times, most recent failure: Lost task 1.3 in stage 26.0 (TID 4765, kafka2, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 3929, in _get_op_def
    return self._op_def_cache[type]
KeyError: 'CaseFoldUTF8'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/load.py", line 890, in load_internal
    ckpt_options, filters)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/load.py", line 132, in __init__
    meta_graph.graph_def.library))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/sav

In [ ]:
# #inference 만

# from tensorflow.keras.models import load_model
# import h5py
# import json

# loaded_model = load_model('bb')
# f = h5py.File('bb', mode='r')

# elephas_conf = json.loads(f.attrs.get('distributed_config'))
# class_name = elephas_conf.get('class_name')
# config = elephas_conf.get('config')
# spark_model = SparkModel(model=model, **config)

# X_test = test.toPandas()['text']
# predictions = spark_model.predict(X_test) # perform inference

## Option 2) Elephas Estimator

In [29]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)
estimator = ElephasEstimator()
estimator.setFeaturesCol("feature2")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(2)
estimator.set_num_workers(3)
estimator.set_custom_objects({'KerasLayer': hub.KerasLayer})
estimator.set_epochs(10)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_4dd63c650289

In [30]:
dl_pipeline = Pipeline(stages=[estimator])

In [32]:
tf.__version__

'2.4.0'

In [31]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(train)
t2 = time.time() - t1
print(t2)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 06:26:51] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:26:54] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:27:00] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 06:27:45] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:27:46] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:27:46] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 06:28:03] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 06:28:03] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:28:04] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:28:20] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [01/Nov/2022 06:28:21] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [01/Nov/2022 06:28:22] "GET /paramet

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 4 times, most recent failure: Lost task 0.3 in stage 22.0 (TID 3524, kafka3, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/pyspark/worker.py", line 377, in main
    process()
  File "/root/spark/python/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.6/dist-packages/elephas/worker.py", line 109, in train
    self.model.fit(x_train, y_train, **self.train_config)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(it

In [ ]:
print(a)

In [ ]:
pred_test = my_dl.transform(test)

In [ ]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='label_index')
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(pred_test)
print(accuracy)

In [28]:
sample_dataset=[
    [5.0,69.0,150.0,0.0,0.0,0.0],
    [3.0,43.0,11.0,0.0,0.0,0.0]]

In [38]:
X_train = train.toPandas()['feature2']
# X_train = np.asarray(X_train).astype(np.int)
Y_train = train.toPandas()['label']
# y_train=np.asarray(y_train).astype(np.int)


In [22]:
XX_train = X_train.apply(lambda x: np.array(x, dtype=np.float))

In [27]:
XX_train.shape

(79985,)

In [28]:
Y_train.shape

(79985,)

In [68]:
Y_train

0        1
1        1
2        1
3        1
4        1
        ..
80105    0
80106    0
80107    0
80108    0
80109    0
Name: label, Length: 80110, dtype: int64

In [69]:
df = pd.read_csv("spam.csv")

In [70]:
df_spam = df[df['Category']=='spam']
df_ham = df[df['Category']=='ham']
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [72]:
X_train

607     XCLUSIVE@CLUBSAISAI 2MOROW 28/5 SOIREE SPECIAL...
3710                 Ok.ok ok..then..whats ur todays plan
2732    Mm feeling sleepy. today itself i shall get th...
4569    hiya hows it going in sunny africa? hope u r a...
3581    Have a lovely night and when you wake up to se...
                              ...                        
4297    Please CALL 08712402578 immediately as there i...
3600               Then wat r u doing now? Busy wif work?
1793    WIN: We have a winner! Mr. T. Foley won an iPo...
1007    Panasonic & BluetoothHdset FREE. Nokia FREE. M...
2826    Congratulations - Thanks to a good friend U ha...
Name: Message, Length: 1120, dtype: object

In [71]:
model.fit(XX_train, Y_train, epochs=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [33]:
#1
model.save_weights('aa')

#2
model.save('bb')

INFO:tensorflow:Assets written to: bb/assets


INFO:tensorflow:Assets written to: bb/assets


In [34]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
loaded_model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [35]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

weight_loaded_model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [38]:
import tensorflow as tf
#1
weight_loaded_model.load_weights('aa')

#2
loaded_model = tf.keras.models.load_model('bb')

In [39]:
y_predicted = loaded_model.predict(X_test)
y_predicted = y_predicted.flatten()

In [40]:
y_predicted

array([0.626552  , 0.71590775, 0.5898657 , 0.5472422 , 0.711022  ,
       0.5268779 , 0.7805311 , 0.4315306 , 0.54762733, 0.71224403,
       0.742162  , 0.6739865 , 0.6467137 , 0.7493478 , 0.5033451 ,
       0.74692804, 0.537749  , 0.7519449 , 0.38838267, 0.73818016,
       0.50813925, 0.51515776, 0.7246484 , 0.6430687 , 0.4697677 ,
       0.6836265 , 0.56010693, 0.5241869 , 0.7043039 , 0.7095687 ,
       0.66718805, 0.6527226 , 0.5277817 , 0.7622446 , 0.5987528 ,
       0.6046336 , 0.64708745, 0.76254636, 0.86459804, 0.46739182,
       0.54351115, 0.73788375, 0.5111824 , 0.7020923 , 0.60011864,
       0.61944103, 0.53094864, 0.4808537 , 0.7680553 , 0.52992004,
       0.58567214, 0.73138547, 0.65863854, 0.4457276 , 0.39486068,
       0.52170473, 0.7092825 , 0.4048063 , 0.71466416, 0.6999085 ,
       0.5090983 , 0.37178427, 0.79798985, 0.71043354, 0.7461667 ,
       0.7263906 , 0.43869263, 0.50580364, 0.6773084 , 0.5050559 ,
       0.42064878, 0.5015567 , 0.533884  , 0.7291204 , 0.78909

In [41]:
y_predicted2 = weight_loaded_model.predict(X_test)
y_predicted2 = y_predicted.flatten()

In [42]:
y_predicted2

array([0.626552  , 0.71590775, 0.5898657 , 0.5472422 , 0.711022  ,
       0.5268779 , 0.7805311 , 0.4315306 , 0.54762733, 0.71224403,
       0.742162  , 0.6739865 , 0.6467137 , 0.7493478 , 0.5033451 ,
       0.74692804, 0.537749  , 0.7519449 , 0.38838267, 0.73818016,
       0.50813925, 0.51515776, 0.7246484 , 0.6430687 , 0.4697677 ,
       0.6836265 , 0.56010693, 0.5241869 , 0.7043039 , 0.7095687 ,
       0.66718805, 0.6527226 , 0.5277817 , 0.7622446 , 0.5987528 ,
       0.6046336 , 0.64708745, 0.76254636, 0.86459804, 0.46739182,
       0.54351115, 0.73788375, 0.5111824 , 0.7020923 , 0.60011864,
       0.61944103, 0.53094864, 0.4808537 , 0.7680553 , 0.52992004,
       0.58567214, 0.73138547, 0.65863854, 0.4457276 , 0.39486068,
       0.52170473, 0.7092825 , 0.4048063 , 0.71466416, 0.6999085 ,
       0.5090983 , 0.37178427, 0.79798985, 0.71043354, 0.7461667 ,
       0.7263906 , 0.43869263, 0.50580364, 0.6773084 , 0.5050559 ,
       0.42064878, 0.5015567 , 0.533884  , 0.7291204 , 0.78909